# Pipeline de Machine Learning pour la prédiction et classification des soudures

In [3]:
# useful libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

plt.style.use("seaborn-v0_8-whitegrid")
sns.set_palette("deep")

In [4]:
# === 1) Chargement du dataset nettoyé
DATA_PATH = Path.cwd().parents[0] / "cleaning" / "outputs" / "weld_quality_clean_from_hf.csv"
df = pd.read_csv(DATA_PATH)
print("[INFO] Shape:", df.shape)
df.head()

[INFO] Shape: (1652, 58)


,Carbon concentration / (weight%),Silicon concentration / (weight%),Manganese concentration / (weight%),Sulphur concentration / (weight%),Phosphorus concentration / (weight%),Nickel concentration / (weight%),Chromium concentration / (weight%),Molybdenum concentration / (weight%),Vanadium concentration / (weight%),Copper concentration / (weight%),...,Niobium concentration / (wt%),Tin concentration / (wt%),Arsenic concentration / (wt%),Antimony concentration / (wt%),Type of weld;_raw,AC or DC_raw,Electrode positive or negative_raw,type_weld_enc,acdc_enc,polarity_enc
0,0.037,0.30,0.65,0.008,0.012,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,MMA,DC,+,0,1.0,1
1,0.037,0.30,0.65,0.008,0.012,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,MMA,DC,+,0,1.0,1
2,0.037,0.30,0.65,0.008,0.012,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,MMA,DC,+,0,1.0,1
3,0.037,0.31,1.03,0.007,0.014,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,MMA,DC,+,0,1.0,1
4,0.037,0.31,1.03,0.007,0.014,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,MMA,DC,+,0,1.0,1


In [5]:
print(df.shape)
print(df.columns)
print(df.isna().sum().sort_values(ascending=False).head(10))

(1652, 58)
Index(['Carbon concentration / (weight%)', 'Silicon concentration / (weight%)',
       'Manganese concentration / (weight%)',
       'Sulphur concentration / (weight%)',
       'Phosphorus concentration / (weight%)',
       'Nickel concentration / (weight%)',
       'Chromium concentration / (weight%)',
       'Molybdenum concentration / (weight%)',
       'Vanadium concentration / (weight%)',
       'Copper concentration / (weight%)', 'Cobalt concentration / (weight%)',
       'Tungsten concentration / (weight%)',
       'Oxygen concentration / parts per million by weight',
       'Titanium concentration / parts per million by weight',
       'Nitrogen concentration / parts per million by weight',
       'Aluminium concentration / parts per million by weight',
       'Boron concentration / parts per million by weight',
       'Niobium concentration / parts per million by weight',
       'Tin concentration / parts per million by weight',
       'Arsenic concentration / parts

In [6]:
df.dtypes.value_counts()

float64    46
object     10
int64       2
Name: count, dtype: int64

In [7]:
# 🎯 Définition des cibles et features
target_cols = [
    'Yield strength / MPa',
    'Ultimate tensile strength / MPa',
    'Elongation / %',
    'Hardness / kgmm-2'
]
feature_cols = [c for c in df.columns if c not in target_cols + ['Weld ID']]

In [8]:
# Sécuriser les cibles en numérique
import numpy as np, pandas as pd
for c in target_cols:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors='coerce')

X_all = df[feature_cols].copy()

In [9]:
# Lignes labellisées (toutes cibles présentes)
label_mask = df[target_cols].notna().all(axis=1)
X_labeled  = X_all.loc[label_mask].copy()
y_labeled  = df.loc[label_mask, target_cols].copy()
X_unlabeled = X_all.loc[~label_mask].copy()

print(f"[INFO] Lignes labellisées : {X_labeled.shape[0]} / {df.shape[0]}")
print(f"[INFO] Lignes non labellisées gardées pour inference : {X_unlabeled.shape[0]}")
assert np.isfinite(y_labeled.to_numpy()).all(), "y contient des valeurs non finies."

[INFO] Lignes labellisées : 54 / 1652
[INFO] Lignes non labellisées gardées pour inference : 1598


In [10]:
# 🔪 Drop des colonnes 100% NaN (sur les labellées) — version stable
# On enlève des features qui sont entièrement NaN côté labellé, et on applique le même drop à l’unlabeled.
all_nan_cols = X_labeled.columns[X_labeled.isna().all(axis=0)]
if len(all_nan_cols) > 0:
    print("[INFO] Colonnes 100% NaN (labellé) — drop:", list(all_nan_cols))
    X_labeled   = X_labeled.drop(columns=all_nan_cols)
    X_unlabeled = X_unlabeled.drop(columns=all_nan_cols, errors="ignore")

[INFO] Colonnes 100% NaN (labellé) — drop: ['Charpy temperature / °C', 'Charpy impact toughness / J', 'Primary ferrite in microstructure / %', 'Ferrite with second phase / %', 'Acicular ferrite / %', 'Martensite / %', 'Ferrite with carbide aggreagate / %']


In [11]:
# =========================
# Helpers + STRATIFIED SPLITS
# =========================
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# --- 9.1) Fonction de label binaire métier
TARGETS = ['Yield strength / MPa','Ultimate tensile strength / MPa','Elongation / %','Hardness / kgmm-2']
def make_label(df_targets, thr, need_at_least=3, hard_thr=None):
    scores = np.zeros(len(df_targets), dtype=int)
    for c in df_targets.columns:
        if c == 'Hardness / kgmm-2' and hard_thr is not None:
            good = (df_targets[c] <= hard_thr)
        else:
            good = (df_targets[c] >= thr[c])
        scores += good.astype(int)
    return (scores >= min(need_at_least, len(df_targets.columns))).astype(int)

# --- 9.2) Pour stratifier, on a besoin d'un y_bin provisoire (pas pour entraîner !)
present_targets_all = [c for c in TARGETS if c in y_labeled.columns]
med_all = {c: y_labeled[c].median() for c in present_targets_all}
hard90_all = np.quantile(y_labeled['Hardness / kgmm-2'].dropna(), 0.90) if 'Hardness / kgmm-2' in present_targets_all else None
y_bin_full = make_label(y_labeled[present_targets_all], med_all, need_at_least=3, hard_thr=hard90_all)

# --- 9.3) Split stratifié (recommandé: 60/20/20 sur les 54 labellées)
X_tr, X_tmp, y_tr, y_tmp, yb_tr, yb_tmp = train_test_split(
    X_labeled, y_labeled, y_bin_full, test_size=0.40, stratify=y_bin_full, random_state=42
)
X_val, X_test, y_val, y_test, yb_val, yb_test = train_test_split(
    X_tmp, y_tmp, yb_tmp, test_size=0.50, stratify=yb_tmp, random_state=42
)

print(f"[SPLIT] train={len(X_tr)} | val={len(X_val)} | test={len(X_test)}  (total={len(X_labeled)})")

# --- 9.4) Définition des seuils métier FINAUX à partir de y_tr uniquement
present_targets_tr = [c for c in TARGETS if c in y_tr.columns]
thr_median_tr = {c: y_tr[c].median() for c in present_targets_tr}
hard_thr_tr   = np.quantile(y_tr['Hardness / kgmm-2'].dropna(), 0.90) if 'Hardness / kgmm-2' in present_targets_tr else None

y_train_bin = make_label(y_tr[present_targets_tr],  thr_median_tr, need_at_least=3, hard_thr=hard_thr_tr)
y_val_bin   = make_label(y_val[present_targets_tr], thr_median_tr, need_at_least=3, hard_thr=hard_thr_tr)
y_test_bin  = make_label(y_test[present_targets_tr],thr_median_tr, need_at_least=3, hard_thr=hard_thr_tr)

print("[LABEL] répartition train/val/test :", np.bincount(y_train_bin), np.bincount(y_val_bin), np.bincount(y_test_bin))

[SPLIT] train=32 | val=11 | test=11  (total=54)
[LABEL] répartition train/val/test : [20 12] [6 5] [8 3]


In [12]:
# =========================
# PREPROCESS (fit sur train + unlabeled uniquement)
# =========================
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

# Colonnes numériques conservées
num_cols = X_tr.select_dtypes(include=[np.number]).columns.tolist()

# 🔧 Aligner X_unlabeled / X_val / X_test sur ces colonnes (ajouter manquantes en NaN)
missing_in_unlabeled = [c for c in num_cols if c not in X_unlabeled.columns]
if missing_in_unlabeled:
    for c in missing_in_unlabeled:
        X_unlabeled[c] = np.nan

for _df in (X_val, X_test):
    miss = [c for c in num_cols if c not in _df.columns]
    if miss:
        for c in miss:
            _df[c] = np.nan

# Re-indexage pour garantir même ordre de colonnes
X_unlabeled = X_unlabeled.reindex(columns=num_cols)
X_val       = X_val.reindex(columns=num_cols)
X_test      = X_test.reindex(columns=num_cols)

# Vues numériques cohérentes
X_tr_num   = X_tr[num_cols].copy()
X_val_num  = X_val[num_cols].copy()
X_test_num = X_test[num_cols].copy()
X_u_num    = X_unlabeled[num_cols].copy()

imp = SimpleImputer(strategy="median")
scaler = StandardScaler()

# Fit imputer/scaler sur (train + unlabeled) — en DataFrame
X_fit = pd.concat([X_tr_num, X_u_num], axis=0)
X_fit_imp = pd.DataFrame(imp.fit_transform(X_fit), columns=num_cols, index=X_fit.index)
# ⬇️ fit scaler sur un DF (avec noms)
scaler.fit(X_fit_imp)

# Transformations cohérentes (toujours DF -> DF)
def pp(df_):
    # imputation -> DF avec mêmes colonnes/index
    arr_imp = imp.transform(df_)
    df_imp  = pd.DataFrame(arr_imp, columns=num_cols, index=df_.index)
    # scaling -> DF avec mêmes colonnes/index
    arr_std = scaler.transform(df_imp)
    df_std  = pd.DataFrame(arr_std, columns=num_cols, index=df_.index)
    # selon ton pipeline en aval, renvoie .values ou laisse en DF
    return df_std.values  # ou return df_std si tu préfères rester en DF jusqu’aux modèles

# et maintenant :
X_tr_pp   = pp(X_tr_num)
X_val_pp  = pp(X_val_num)
X_test_pp = pp(X_test_num)
X_u_pp    = pp(X_u_num)

In [13]:
# =========================
# (Option) LABEL SPREADING — robustifié
# =========================
import numpy as np
from sklearn.semi_supervised import LabelSpreading, LabelPropagation
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.metrics import pairwise_distances

X_ssl_all = np.vstack([X_tr_pp, X_u_pp])
y_ssl = np.full(X_ssl_all.shape[0], -1, dtype=int)
y_ssl[:len(X_tr_pp)] = y_train_bin

def median_gamma(X, max_samples=1000):
    # gamma ~ 1 / (median distance)^2
    n = X.shape[0]
    if n > max_samples:
        idx = np.random.RandomState(42).choice(n, size=max_samples, replace=False)
        Xs = X[idx]
    else:
        Xs = X
    # pairwise on a subset of rows to estimate median distance
    d = pairwise_distances(Xs, Xs, metric="euclidean")
    d = d[np.triu_indices_from(d, k=1)]
    med = np.median(d[d > 0]) if np.any(d > 0) else 1.0
    return 1.0 / (med**2 if med > 0 else 1.0)

def safe_auc(y_true, scores):
    s = np.nan_to_num(scores, nan=0.5, posinf=1.0, neginf=0.0)
    # si toutes les valeurs sont identiques, AUC n’a pas de sens
    if np.unique(s).size < 2:
        return float("nan")
    try:
        return roc_auc_score(y_true, s)
    except Exception:
        return float("nan")

def eval_ls(model, X_val_pp, y_val_bin):
    p = model.predict_proba(X_val_pp)[:, 1]
    p = np.nan_to_num(p, nan=0.5, posinf=1.0, neginf=0.0)
    yhat = (p >= 0.5).astype(int)
    f1 = f1_score(y_val_bin, yhat)
    auc = safe_auc(y_val_bin, p)
    return f1, auc, p

# grille robuste : gamma auto + plusieurs k
gamma_auto = median_gamma(X_ssl_all, max_samples=1200)
cands = [
    ("rbf", gamma_auto), ("rbf", gamma_auto/2), ("rbf", gamma_auto*2),
    ("knn", 10), ("knn", 20), ("knn", 30), ("knn", 50)
]

best_ls, best = None, (-np.inf,)

for kernel, param in cands:
    if kernel == "rbf":
        ls = LabelSpreading(kernel="rbf", gamma=float(param), alpha=0.2, max_iter=30, tol=1e-3)
    else:
        k = int(param)
        # k au moins 2 et pas plus que n-1
        k = max(2, min(k, X_ssl_all.shape[0]-1))
        ls = LabelSpreading(kernel="knn", n_neighbors=k, alpha=0.2, max_iter=30, tol=1e-3)

    ls.fit(X_ssl_all, y_ssl)
    f1, auc, p = eval_ls(ls, X_val_pp, y_val_bin)
    score = (f1, np.nan_to_num(auc))
    if score > best:
        best, best_ls = score, ls

print(f"[LS] best on val: F1={best[0]:.3f} | AUC={best[1]:.3f}")

# 🔁 fallback : si l’AUC reste NaN/0 avec Spreading, tente LabelPropagation (hard clamping) pour info
try:
    lp = LabelPropagation(kernel="knn", n_neighbors=20, max_iter=30, tol=1e-3)
    lp.fit(X_ssl_all, y_ssl)
    f1_lp, auc_lp, _ = eval_ls(lp, X_val_pp, y_val_bin)
    print(f"[LP] (check) F1={f1_lp:.3f} | AUC={auc_lp:.3f}")
except Exception as e:
    print("[LP] skip:", e)

[LS] best on val: F1=1.000 | AUC=1.000
[LP] (check) F1=0.889 | AUC=1.000


C:\Users\Guillaume PORET\AppData\Local\Programs\Python\Python310\Lib\site-packages\sklearn\semi_supervised\_label_propagation.py:231: RuntimeWarning: invalid value encountered in divide
  probabilities /= normalizer
C:\Users\Guillaume PORET\AppData\Local\Programs\Python\Python310\Lib\site-packages\sklearn\semi_supervised\_label_propagation.py:231: RuntimeWarning: invalid value encountered in divide
  probabilities /= normalizer
C:\Users\Guillaume PORET\AppData\Local\Programs\Python\Python310\Lib\site-packages\sklearn\semi_supervised\_label_propagation.py:324: ConvergenceWarning: max_iter=30 was reached without convergence.
  warnings.warn(
C:\Users\Guillaume PORET\AppData\Local\Programs\Python\Python310\Lib\site-packages\sklearn\semi_supervised\_label_propagation.py:231: RuntimeWarning: invalid value encountered in divide
  probabilities /= normalizer


In [14]:
# =========================
# TEACHER–STUDENT avec quotas
# =========================
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, roc_auc_score, classification_report, accuracy_score, confusion_matrix

def add_gaussian_noise(X, sigma=0.01, seed=42):
    rs = np.random.RandomState(seed)
    return X + rs.normal(0.0, sigma, size=X.shape)

def class_weight_ratio(y):
    pos = np.sum(y==1); neg = np.sum(y==0)
    return max(1.0, (neg / max(1, pos)))

def make_rf(seed=42):
    return RandomForestClassifier(
        n_estimators=900, max_depth=None, min_samples_leaf=1,
        class_weight="balanced_subsample", n_jobs=-1, random_state=seed
    )

def make_xgb(y_train=None, seed=42):
    try:
        from xgboost import XGBClassifier
    except Exception as e:
        print(f"[XGB] indisponible: {e}")
        return None
    spw = class_weight_ratio(y_train) if y_train is not None else 1.0
    return XGBClassifier(
        n_estimators=1000, max_depth=6, learning_rate=0.05,
        subsample=0.85, colsample_bytree=0.85,
        reg_lambda=1.0, reg_alpha=0.0,
        random_state=seed, n_jobs=-1, tree_method="hist",
        eval_metric="logloss", scale_pos_weight=spw
    )

def teacher_student_quota(model_maker, X_l, y_l, X_u,
                          thresholds=(0.99,0.97,0.95,0.93,0.90),
                          max_add_frac=0.10,  # 10% du pool par itération
                          aug_sigma=0.01, seed=42):
    teacher = model_maker(seed=seed)
    teacher.fit(X_l, y_l)

    added = np.zeros(len(X_u), dtype=bool)
    total = 0
    for it, thr in enumerate(thresholds, start=1):
        pool_idx = np.where(~added)[0]
        if pool_idx.size == 0: break

        proba = teacher.predict_proba(X_u[pool_idx])[:,1]
        conf = np.maximum(proba, 1.0 - proba)
        yhat = (proba>=0.5).astype(int)
        keep = conf >= thr
        if not np.any(keep):
            continue
        idx_keep = pool_idx[keep]

        # quotas par classe
        pos_idx = idx_keep[yhat[keep]==1]
        neg_idx = idx_keep[yhat[keep]==0]
        cap = int(max_add_frac * len(pool_idx))
        half = max(1, cap//2)
        pos_take = pos_idx[:half]
        neg_take = neg_idx[:half]
        add_idx = np.concatenate([pos_take, neg_take])
        if add_idx.size == 0:
            continue

        X_pl = X_u[add_idx]
        y_pl = teacher.predict(X_pl)  # labels durs pour stabiliser
        # Consistency par bruit
        X_aug = np.vstack([X_l, add_gaussian_noise(X_l, aug_sigma, seed+it),
                           X_pl, add_gaussian_noise(X_pl, aug_sigma, seed+100+it)])
        y_aug = np.concatenate([y_l, y_l, y_pl, y_pl])

        student = model_maker(seed=seed+it)
        student.fit(X_aug, y_aug)
        teacher = student
        added[add_idx] = True
        total += add_idx.size
        print(f"[TS] it{it} thr={thr:.2f} +{add_idx.size} (pos={len(pos_take)} / neg={len(neg_take)}) total={total}")

    return teacher, dict(added=int(total), remain=int((~added).sum()))

In [15]:
# =========================
# Entraînement RF & XGB (Teacher–Student) + ensemble
# =========================
rf_teacher, rf_stats = teacher_student_quota(make_rf,  X_tr_pp, y_train_bin, X_u_pp,
                                             thresholds=(0.99,0.97,0.95,0.93,0.90),
                                             max_add_frac=0.10, aug_sigma=0.01, seed=42)
print("[RF] Stats:", rf_stats)

xgb_teacher, xgb_stats = teacher_student_quota(make_xgb, X_tr_pp, y_train_bin, X_u_pp,
                                               thresholds=(0.99,0.97,0.95,0.93,0.90),
                                               max_add_frac=0.10, aug_sigma=0.01, seed=777)
print("[XGB] Stats:", xgb_stats)

from sklearn.metrics import classification_report

candidates = {}

# RF → val
if rf_teacher is not None:
    pr = rf_teacher.predict_proba(X_val_pp)[:,1]
    yr = (pr>=0.5).astype(int)
    candidates["TS_RF"] = dict(f1=f1_score(y_val_bin, yr), auc=roc_auc_score(y_val_bin, pr), proba=pr)

# XGB → val
if xgb_teacher is not None:
    try:
        px = xgb_teacher.predict_proba(X_val_pp)[:,1]
        yx = (px>=0.5).astype(int)
        candidates["TS_XGB"] = dict(f1=f1_score(y_val_bin, yx), auc=roc_auc_score(y_val_bin, px), proba=px)
    except Exception as e:
        print("[XGB] predict_proba indisponible:", e)

# Ensemble (poids AUC sur val)
if ("TS_RF" in candidates) and ("TS_XGB" in candidates):
    auc_rf = candidates["TS_RF"]["auc"]
    auc_xg = candidates["TS_XGB"]["auc"]
    s = max(auc_rf+auc_xg, 1e-9)
    w_rf, w_xg = auc_rf/s, auc_xg/s
    p_ens = w_rf*candidates["TS_RF"]["proba"] + w_xg*candidates["TS_XGB"]["proba"]
    y_ens = (p_ens>=0.5).astype(int)
    candidates["TS_Ensemble"] = dict(f1=f1_score(y_val_bin, y_ens), auc=roc_auc_score(y_val_bin, p_ens),
                                     proba=p_ens, weights=(w_rf, w_xg))

# Sélection champion sur validation (F1 puis AUC)
champ_name, champ_stats = max(candidates.items(), key=lambda kv: (kv[1]["f1"], kv[1]["auc"]))

print(f"[🏆 Champion (val)] {champ_name} — F1={champ_stats['f1']:.3f} | AUC={champ_stats['auc']:.3f}")

[RF] Stats: {'added': 0, 'remain': 1598}
[TS] it1 thr=0.99 +2 (pos=2 / neg=0) total=2
[TS] it2 thr=0.97 +111 (pos=79 / neg=32) total=113
[TS] it3 thr=0.95 +148 (pos=74 / neg=74) total=261
[TS] it4 thr=0.93 +110 (pos=44 / neg=66) total=371
[TS] it5 thr=0.90 +67 (pos=6 / neg=61) total=438
[XGB] Stats: {'added': 438, 'remain': 1160}
[🏆 Champion (val)] TS_Ensemble — F1=0.833 | AUC=0.867


In [16]:
from sklearn.metrics import classification_report

def print_clf_report(name, proba, y_true, thr=0.5):
    y_pred = (proba >= thr).astype(int)
    print(f"\n--- {name} (thr={thr:.2f}) ---")
    print(classification_report(y_true, y_pred, digits=3))

if "TS_RF" in candidates:       print_clf_report("TS_RF", candidates["TS_RF"]["proba"], y_val_bin)
if "TS_XGB" in candidates:      print_clf_report("TS_XGB", candidates["TS_XGB"]["proba"], y_val_bin)
if "TS_Ensemble" in candidates: print_clf_report("TS_Ensemble", candidates["TS_Ensemble"]["proba"], y_val_bin)


--- TS_RF (thr=0.50) ---
              precision    recall  f1-score   support

           0      0.714     0.833     0.769         6
           1      0.750     0.600     0.667         5

    accuracy                          0.727        11
   macro avg      0.732     0.717     0.718        11
weighted avg      0.731     0.727     0.723        11


--- TS_XGB (thr=0.50) ---
              precision    recall  f1-score   support

           0      1.000     0.500     0.667         6
           1      0.625     1.000     0.769         5

    accuracy                          0.727        11
   macro avg      0.812     0.750     0.718        11
weighted avg      0.830     0.727     0.713        11


--- TS_Ensemble (thr=0.50) ---
              precision    recall  f1-score   support

           0      1.000     0.667     0.800         6
           1      0.714     1.000     0.833         5

    accuracy                          0.818        11
   macro avg      0.857     0.833     0.817

In [2]:
!pip install statsmodels

   ---------------------------------------- 0.0/9.6 MB ? eta -:--:--
   ------------- -------------------------- 3.1/9.6 MB 23.1 MB/s eta 0:00:01
   ------------- -------------------------- 3.1/9.6 MB 23.1 MB/s eta 0:00:01
   ------------- -------------------------- 3.1/9.6 MB 23.1 MB/s eta 0:00:01
   -------------- ------------------------- 3.4/9.6 MB 4.5 MB/s eta 0:00:02
   ---------------------- ----------------- 5.5/9.6 MB 5.2 MB/s eta 0:00:01
   ----------------------- ---------------- 5.8/9.6 MB 5.4 MB/s eta 0:00:01
   ----------------------- ---------------- 5.8/9.6 MB 5.4 MB/s eta 0:00:01
   ----------------------- ---------------- 5.8/9.6 MB 5.4 MB/s eta 0:00:01
   --------------------------------- ------ 8.1/9.6 MB 4.3 MB/s eta 0:00:01
   --------------------------------- ------ 8.1/9.6 MB 4.3 MB/s eta 0:00:01
   --------------------------------- ------ 8.1/9.6 MB 4.3 MB/s eta 0:00:01
   --------------------------------- ------ 8.1/9.6 MB 4.3 MB/s eta 0:00:01
   -------------


[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [17]:
# ============================================
# 🔍 DIAG ENSEMBLE — AVANT CALIBRATION (sur VALIDATION)
# ============================================

from sklearn.metrics import brier_score_loss

# Probas brutes sur la validation
pr   = candidates["TS_RF"]["proba"]
px   = candidates["TS_XGB"]["proba"]
p_ens= candidates["TS_Ensemble"]["proba"]

# Prédictions binaires (thr=0.5 pour le diag)
yr   = (pr>=0.5).astype(int)
yx   = (px>=0.5).astype(int)
yens = (p_ens>=0.5).astype(int)

# 1) Taux de désaccord RF vs XGB
disagree = float(np.mean(yr != yx))

# 2) Corrélation des erreurs RF vs XGB
err_corr = float(np.corrcoef((yr!=y_val_bin), (yx!=y_val_bin))[0,1])

# 3) Gain F1 de l’ensemble vs chaque modèle
d_rf = f1_score(y_val_bin, yens) - f1_score(y_val_bin, yr)
d_xg = f1_score(y_val_bin, yens) - f1_score(y_val_bin, yx)

# 4) ΔAUC de l’ensemble
auc_rf  = roc_auc_score(y_val_bin, pr)
auc_xgb = roc_auc_score(y_val_bin, px)
auc_ens = roc_auc_score(y_val_bin, p_ens)

# 5) Brier (baseline non calibrée)
brier_uncal = brier_score_loss(y_val_bin, p_ens)

print("=== DIAG ENSEMBLE (avant calibration) ===")
print(f"Taux de désaccord RF vs XGB     : {disagree:.3f}")
print(f"Corrélation des erreurs         : {err_corr:.3f}")
print(f"ΔF1 ensemble vs RF              : {d_rf:+.3f}")
print(f"ΔF1 ensemble vs XGB             : {d_xg:+.3f}")
print(f"AUC RF / XGB / ENS              : {auc_rf:.3f} | {auc_xgb:.3f} | {auc_ens:.3f}")
print(f"Brier (ensemble, uncal)         : {brier_uncal:.4f}")

# 6) McNemar (significativité des écarts)
try:
    from statsmodels.stats.contingency_tables import mcnemar

    def mcnemar_p(yA, yB):
        b01 = int(np.sum((yA==y_val_bin) & (yB!=y_val_bin)))  # A correct, B faux
        b10 = int(np.sum((yA!=y_val_bin) & (yB==y_val_bin)))  # A faux, B correct
        res = mcnemar([[0, b01],[b10, 0]], exact=True)
        return res.pvalue, b01, b10

    p_rf, b01_rf, b10_rf = mcnemar_p(yr, yens)
    p_xg, b01_xg, b10_xg = mcnemar_p(yx, yens)

    print(f"McNemar RF vs Ensemble          : p={p_rf:.3f} (RF-corr/Ens-faux={b01_rf}, RF-faux/Ens-corr={b10_rf})")
    print(f"McNemar XGB vs Ensemble         : p={p_xg:.3f} (XGB-corr/Ens-faux={b01_xg}, XGB-faux/Ens-corr={b10_xg})")
except Exception as e:
    print(f"McNemar indisponible ({e}). Installe 'statsmodels' si besoin.")

=== DIAG ENSEMBLE (avant calibration) ===
Taux de désaccord RF vs XGB     : 0.364
Corrélation des erreurs         : 0.083
ΔF1 ensemble vs RF              : +0.167
ΔF1 ensemble vs XGB             : +0.064
AUC RF / XGB / ENS              : 0.900 | 0.800 | 0.867
Brier (ensemble, uncal)         : 0.1618
McNemar RF vs Ensemble          : p=1.000 (RF-corr/Ens-faux=1, RF-faux/Ens-corr=2)
McNemar XGB vs Ensemble         : p=1.000 (XGB-corr/Ens-faux=0, XGB-faux/Ens-corr=1)


In [18]:
# =========================
# Calibration isotonic SANS 'cv=prefit' (compatible ≥ 1.6)
# =========================
import numpy as np
from sklearn.isotonic import IsotonicRegression

class IsoCalibrated:
    """
    Wrappe un classifieur déjà entraîné et applique une calibration isotonic
    sur sa probabilité p(y=1|x).
    """
    def __init__(self, base_model, iso_reg):
        self.base_model = base_model
        self.iso = iso_reg

    def predict_proba(self, X):
        p = self.base_model.predict_proba(X)[:, 1]
        p_cal = self.iso.transform(p)
        p_cal = np.clip(p_cal, 0.0, 1.0)
        return np.vstack([1.0 - p_cal, p_cal]).T

def fit_isotonic_on_val(prefit_model, X_val_pp, y_val_bin):
    # proba brutes du modèle pré-entrainé
    p_val = prefit_model.predict_proba(X_val_pp)[:, 1]
    # fit isotonic: monotone, clip hors bornes
    iso = IsotonicRegression(out_of_bounds="clip")
    iso.fit(p_val, y_val_bin)
    return IsoCalibrated(prefit_model, iso)

# ---- Choix du modèle à calibrer selon le champion ----
calibrated = None
ensemble_weights = None

if champ_name == "TS_RF":
    calibrated = fit_isotonic_on_val(rf_teacher, X_val_pp, y_val_bin)

elif champ_name == "TS_XGB":
    calibrated = fit_isotonic_on_val(xgb_teacher, X_val_pp, y_val_bin)

elif champ_name == "TS_Ensemble":
    # on calibre chaque composant séparément puis on fait un blend pondéré (sur proba déjà calibrées)
    rf_cal = fit_isotonic_on_val(rf_teacher,  X_val_pp, y_val_bin)
    xg_cal = fit_isotonic_on_val(xgb_teacher, X_val_pp, y_val_bin)
    w_rf, w_xg = candidates["TS_Ensemble"]["weights"]
    ensemble_weights = (w_rf, w_xg)

    class CalEnsemble:
        def __init__(self, rf, xg, w_rf, w_xg):
            self.rf = rf; self.xg = xg; self.w_rf = float(w_rf); self.w_xg = float(w_xg)
        def predict_proba(self, X):
            pr = self.rf.predict_proba(X)[:, 1]   # déjà calibrées (isotonic)
            px = self.xg.predict_proba(X)[:, 1]
            p  = self.w_rf*pr + self.w_xg*px
            p  = np.clip(p, 0.0, 1.0)
            return np.vstack([1.0 - p, p]).T

    calibrated = CalEnsemble(rf_cal, xg_cal, w_rf, w_xg)

assert calibrated is not None, "Calibration introuvable."

In [24]:
# ============================================
# ✅ CONTRÔLE POST-CALIBRATION (sur VALIDATION)
# ============================================

# Proba non calibrées du champion (si champion ≠ ensemble, adapte la clé)
p_val_uncal = candidates[champ_name]["proba"] if champ_name in candidates else p_ens
p_val_cal   = calibrated.predict_proba(X_val_pp)[:,1]

# 1) Brier avant/après (doit baisser)
brier_uncal = brier_score_loss(y_val_bin, p_val_uncal)
brier_cal   = brier_score_loss(y_val_bin, p_val_cal)

# 2) AUC avant/après (peut bouger un peu ; isotonic préserve l’ordre local)
auc_uncal = roc_auc_score(y_val_bin, p_val_uncal)
auc_cal   = roc_auc_score(y_val_bin, p_val_cal)

print("=== POST-CALIBRATION (validation) ===")
print(f"Brier uncal / cal                : {brier_uncal:.4f} → {brier_cal:.4f}")
print(f"AUC   uncal / cal                : {auc_uncal:.3f}  → {auc_cal:.3f}")

=== POST-CALIBRATION (validation) ===
Brier uncal / cal                : 0.1618 → 0.0772
AUC   uncal / cal                : 0.867  → 0.950


In [20]:
# =========================
# Seuil optimal sur validation
# =========================
from sklearn.metrics import f1_score, roc_auc_score

p_val = calibrated.predict_proba(X_val_pp)[:,1]
ths = np.linspace(0.1, 0.9, 81)
best_t, best_f1 = 0.5, -1
for t in ths:
    f1 = f1_score(y_val_bin, (p_val>=t).astype(int))
    if f1 > best_f1:
        best_f1, best_t = f1, t

margin = 0.10  # zone grise
p_good = float(np.clip(best_t, 0.3, 0.8))
p_bad  = float(np.clip(best_t - margin, 0.1, p_good - 1e-3))

print(f"[Seuil (val)] t*={best_t:.3f} (F1={best_f1:.3f}) → p_good={p_good:.3f} | p_bad={p_bad:.3f}")

[Seuil (val)] t*=0.290 (F1=0.909) → p_good=0.300 | p_bad=0.190


In [21]:
# =========================
# Évaluation finale sur TEST
# =========================
from sklearn.metrics import accuracy_score, confusion_matrix

p_test = calibrated.predict_proba(X_test_pp)[:,1]
y_test_pred = (p_test >= p_good).astype(int)

print("\n=== Test (final) — Champion calibré ===")
print(classification_report(y_test_bin, y_test_pred, digits=3))
print("Accuracy :", accuracy_score(y_test_bin, y_test_pred))
print("F1-score :", f1_score(y_test_bin, y_test_pred))
try:
    print("ROC-AUC  :", roc_auc_score(y_test_bin, p_test))
except ValueError:
    pass
print("Confusion :\n", confusion_matrix(y_test_bin, y_test_pred))


=== Test (final) — Champion calibré ===
              precision    recall  f1-score   support

           0      1.000     0.625     0.769         8
           1      0.500     1.000     0.667         3

    accuracy                          0.727        11
   macro avg      0.750     0.812     0.718        11
weighted avg      0.864     0.727     0.741        11

Accuracy : 0.7272727272727273
F1-score : 0.6666666666666666
ROC-AUC  : 0.75
Confusion :
 [[5 3]
 [0 3]]


In [25]:
# =========================
# Inférence sur TOUTES les soudures + export + bundle
# =========================
from pathlib import Path
from datetime import datetime
import joblib

# Recompose toutes les lignes (unlabeled PUIS labeled)
X_all_raw_num = pd.concat([X_unlabeled[num_cols], X_labeled[num_cols]], axis=0)
X_all_pp = pp(X_all_raw_num)  # même imp/scale

proba_all = calibrated.predict_proba(X_all_pp)[:,1]

def decide(p):
    if p >= p_good: return "GOOD"
    if p <= p_bad:  return "BAD"
    return "UNCERTAIN"

df_final = pd.DataFrame({
    "GoodWeld_proba_SSL": proba_all,
    "GoodWeld_label_SSL": (proba_all >= p_good).astype(int)
}, index=X_all_raw_num.index)
df_final["Decision_SSL"] = df_final["GoodWeld_proba_SSL"].apply(decide)

df_final_sorted = df_final.sort_values("GoodWeld_proba_SSL", ascending=False).copy()
df_final_sorted.insert(0, "Rank", np.arange(1, len(df_final_sorted)+1))
df_final_sorted["GoodWeld_proba_pct"] = (100*df_final_sorted["GoodWeld_proba_SSL"]).round(2)

# joindre meta utiles
meta_cols = [c for c in ["Weld ID","Type of weld;_raw","AC or DC_raw","Electrode positive or negative_raw",
                         "Current / A","Voltage / V","Heat input / kJmm-1","Interpass temperature / °C"] if c in df.columns]
df_meta = df.loc[df_final_sorted.index, meta_cols] if meta_cols else pd.DataFrame(index=df_final_sorted.index)
final_export = pd.concat([df_meta, df_final_sorted], axis=1)

# exports
OUT_DIR = Path.cwd() / "outputs"
OUT_DIR.mkdir(parents=True, exist_ok=True)
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

csv_path = OUT_DIR / f"weld_predictions_ssl_{champ_name}_CAL_{timestamp}.csv"
final_export.to_csv(csv_path, index=True)
print(f"[✅] Export : {csv_path}")
print("\n📊 Répartition finale :")
print(final_export["Decision_SSL"].value_counts())

# bundle (sauvegarde prod)
bundle = {
    "model_type": champ_name,
    "model": calibrated,
    "imputer": imp,
    "scaler": scaler,
    "num_cols": num_cols,
    "p_good": p_good,
    "p_bad": p_bad,
    "meta_cols": meta_cols
}
# si ensemble calibré, garde les poids
if champ_name == "TS_Ensemble":
    bundle["ensemble_weights"] = ensemble_weights

bundle_path = OUT_DIR / f"ssl_champion_bundle_{champ_name}_{timestamp}.joblib"
joblib.dump(bundle, bundle_path)
print(f"[💾] Bundle sauvegardé : {bundle_path}")

[✅] Export : C:\Users\Guillaume PORET\PycharmProjects\pythonProject\Central\weld-quality\V2 test semi supervides learning\src\processing\outputs\weld_predictions_ssl_TS_Ensemble_CAL_20251028_155732.csv

📊 Répartition finale :
Decision_SSL
BAD          1229
GOOD          353
UNCERTAIN      70
Name: count, dtype: int64
[💾] Bundle sauvegardé : C:\Users\Guillaume PORET\PycharmProjects\pythonProject\Central\weld-quality\V2 test semi supervides learning\src\processing\outputs\ssl_champion_bundle_TS_Ensemble_20251028_155732.joblib
